In [1]:
import torch
from transformers import BertTokenizer, BertForMultipleChoice, AdamW
from torch.utils.data import DataLoader, Dataset
import pandas as pd

c:\Users\Alvin Lui\anaconda3\envs\eedi\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv("../source/train.csv")
test = pd.read_csv("../source/test.csv")
mis_map = pd.read_csv("../source/misconception_mapping.csv")
sub = pd.read_csv("../source/sample_submission.csv")

### Data Preparation

In [3]:
x_train = train[test.columns]
y_train = train[train.columns.difference(test.columns)]

### Code Implementation

In [4]:
# # Example MCQ dataset (replace with your actual data)
# data = {
#     'question': ['What is the capital of France?', 'Which planet is known as the Red Planet?'],
#     'options': [['London', 'Paris', 'Berlin', 'Madrid'], 
#                 ['Earth', 'Mars', 'Jupiter', 'Saturn']],
#     'correct_answer': [1, 1]  # Correct answer index (0-based)
# }

# # Convert to DataFrame
# df = pd.DataFrame(data)

In [5]:
mapping = {'A': 0,
           'B': 1,
           'C': 2,
           'D': 3}

data = {
    'question': x_train['QuestionText'],
    'options': pd.Series([list(x) for x in zip(x_train['AnswerAText'],
                                               x_train['AnswerBText'],
                                               x_train['AnswerCText'],
                                               x_train['AnswerDText'])]),
    'correct_answer': x_train['CorrectAnswer'].map(mapping)
}

df = pd.DataFrame(data)

In [6]:
# Custom Dataset class for MCQ
class MCQDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=64):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        question = self.df.iloc[idx]['question']
        options = self.df.iloc[idx]['options']
        correct_answer = self.df.iloc[idx]['correct_answer']
        
        # Tokenize the question with each option
        inputs = [self.tokenizer(question, option, truncation=True, padding='max_length', max_length=self.max_len, return_tensors="pt") for option in options]

        # Stack tokenized inputs
        input_ids = torch.cat([x['input_ids'] for x in inputs])
        attention_mask = torch.cat([x['attention_mask'] for x in inputs])

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(correct_answer)
        }

In [ ]:
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Initialize dataset and dataloader
dataset = MCQDataset(df, tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Initialize model for multiple choice
model = BertForMultipleChoice.from_pretrained('bert-base-uncased')
optimizer = AdamW(model.parameters(), lr=5e-5)

# Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    print(f'Epoch {epoch+1}, Loss: {avg_loss}')

c:\Users\Alvin Lui\anaconda3\envs\eedi\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Alvin Lui\anaconda3\envs\eedi\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning

Epoch 1, Loss: 1.392136466439395


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
# Model evaluation (you can expand this for more complex metrics)
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=1)
            correct += (predictions == labels).sum().item()
            total += len(labels)
    return correct / total

# Evaluate the model
accuracy = evaluate(model, dataloader)
print(f'Accuracy: {accuracy * 100:.2f}%')



### Reference

https://www.kaggle.com/competitions/eedi-mining-misconceptions-in-mathematics